<a href="https://colab.research.google.com/github/srinivasrdhkrshnn/CS6910_Assignment_1/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
src_url = "https://storage.googleapis.com/wandb_datasets/nature_12K.zip"
src_zip = "nature_12K.zip"
DATA_TRAIN_SRC = "inaturalist_12K/train" 
DATA_TEST_SRC = "inaturalist_12K/val"
TRAIN_IMAGES_PER_LABEL = 1000
TEST_IMAGES_PER_LABEL = 200
BALANCED_SPLITS = {"train" : 900, "val" : 100}

In [8]:
%%capture
!curl -SL $src_url > $src_zip
!unzip $src_zip

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle

!pip3 install tensorflow -qqq
!pip3 install wandb -qqq
import wandb
!wandb login
from wandb.keras import WandbCallback

wandb: Currently logged in as: rohithd (use `wandb login --relogin` to force relogin)


In [ ]:
PROJECT_NAME = "CS6910 ASSIGNMENT 2"

**Upload Raw Train Data**

In [10]:
# source directory for all raw train data
SRC_TRAIN = DATA_TRAIN_SRC
# number of images per class label
# the total number of images is 10X this (10 classes)
TOTAL_IMAGES = TRAIN_IMAGES_PER_LABEL * 10
PREFIX_1 = "train" # convenient for tracking local data

TRAIN_RAW_DATA_AT = "_".join([PREFIX_1, "raw_data", str(TOTAL_IMAGES)])
run = wandb.init(project=PROJECT_NAME, job_type="upload")

# create an artifact for all the raw data
raw_data_at = wandb.Artifact(TRAIN_RAW_DATA_AT, type="raw_data")

# SRC_DIR contains 10 folders, one for each of 10 class labels
# each folder contains images of the corresponding class
labels = os.listdir(SRC_TRAIN)
for l in labels:
  imgs_per_label = os.path.join(SRC_TRAIN, l)
  if os.path.isdir(imgs_per_label):
    imgs = os.listdir(imgs_per_label)
    # randomize the order
    shuffle(imgs)
    img_file_ids = imgs[:TRAIN_IMAGES_PER_LABEL]
    for f in img_file_ids:
      file_path = os.path.join(SRC_TRAIN, l, f)
      # add file to artifact by full path
      raw_data_at.add_file(file_path, name=l + "/" + f)

# save artifact to W&B
run.log_artifact(raw_data_at)
run.finish()

**Upload Raw Test Data**

In [11]:
# source directory for all raw train data
SRC_TEST = DATA_TEST_SRC
# number of images per class label
# the total number of images is 10X this (10 classes)
TOTAL_IMAGES = TEST_IMAGES_PER_LABEL * 10
PREFIX_2 = "test" # convenient for tracking local data

TEST_RAW_DATA_AT = "_".join([PREFIX_2, "raw_data", str(TOTAL_IMAGES)])
run = wandb.init(project=PROJECT_NAME, job_type="upload")

# create an artifact for all the raw data
raw_data_at = wandb.Artifact(TEST_RAW_DATA_AT, type="raw_data")

# SRC_DIR contains 10 folders, one for each of 10 class labels
# each folder contains images of the corresponding class
labels = os.listdir(SRC_TEST)
for l in labels:
  imgs_per_label = os.path.join(SRC_TEST, l)
  if os.path.isdir(imgs_per_label):
    imgs = os.listdir(imgs_per_label)
    # randomize the order
    shuffle(imgs)
    img_file_ids = imgs[:TEST_IMAGES_PER_LABEL]
    for f in img_file_ids:
      file_path = os.path.join(SRC_TEST, l, f)
      # add file to artifact by full path
      raw_data_at.add_file(file_path, name=l + "/" + f)

# save artifact to W&B
run.log_artifact(raw_data_at)
run.finish()

**Split Train Data into Train and Validation**

In [12]:
run = wandb.init(project=PROJECT_NAME, job_type="data_split")

# find the most recent ("latest") version of the full raw data
# you can of course pass around programmatic aliases and not string literals
data_at = run.use_artifact(TRAIN_RAW_DATA_AT + ":latest")
# download it locally (for illustration purposes/across hardware; you can
# also sync/version artifacts by reference)
data_dir = data_at.download()

# create balanced train, val, test splits
# each count is the number of images per label
DATA_SPLITS = BALANCED_SPLITS

ats = {}
# wrap artifacts in dictionary for convenience
for split, count in DATA_SPLITS.items():
  ats[split] = wandb.Artifact("_".join([PREFIX_1, split, "data", str(count*10)]), 
                              "_".join([split, "data"]))

labels = os.listdir(data_dir)
for l in labels:
  if l.startswith("."): # skip non-label file
    continue
  imgs_per_label = os.listdir(os.path.join(data_dir, l))
  shuffle(imgs_per_label)
  start_id = 0
  for split, count in DATA_SPLITS.items():
    # take a subset
    split_imgs = imgs_per_label[start_id:start_id+count]
    for img_file in split_imgs:
      full_path = os.path.join(data_dir, l, img_file)
      # add file to artifact by full path
      # note: pass the label to the name parameter to retain it in
      # the data structure 
      ats[split].add_file(full_path, name = os.path.join(l, img_file))
    start_id += count

# save all three artifacts to W&B
# note: yes, in this example, we are cheating and have labels for the "val" data ;)
for split, artifact in ats.items():
  run.log_artifact(artifact)

run.finish()

wandb: Downloading large artifact train_raw_data_10000:latest, 3044.53MB. 10000 files... Done. 0:0:0


**Default Configuration**

In [13]:
MODEL_NAME = "CNN"
FINAL_MODEL_DIR = "trained_CNN"

config_defaults = {
    "img_size" : 229,
    "batch_size" : 100,
    "n_classes" : 10,
    "n_filters" : [16,32,64,64,128],
    "filter_size" : [3,3,3,3,3],
    "fc_size" : 400,
    "drop_out" : 0.5,
    "augmentation" : 1,
    "batch_normalize" : 1 
}

n_train = BALANCED_SPLITS["train"] * 10
n_val = BALANCED_SPLITS["val"] * 10
n_epochs = 5 

**Define and Compile Model**

In [14]:
def cnn_model(img_size,n_filters,filter_size,fc_size,drop_out,n_classes):

   model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(n_filters[0], (filter_size[0],filter_size[0]), activation='relu', input_shape=(img_size,img_size,3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.Conv2D(n_filters[1], (filter_size[1],filter_size[1]), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2,2),

        tf.keras.layers.Conv2D(n_filters[2], (filter_size[2],filter_size[2]), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2,2),

        tf.keras.layers.Conv2D(n_filters[3], (filter_size[3],filter_size[3]), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2,2),

        tf.keras.layers.Conv2D(n_filters[4], (filter_size[4],filter_size[4]), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(2,2),

        tf.keras.layers.Dropout(drop_out),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(fc_size, activation='relu'),
        tf.keras.layers.Dense(n_classes,activation="softmax")
    ])

   model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
   model.summary()
   return model

   

**Train Model**

In [15]:
def train():
  
  config_defaults = {
      "img_size" : 229,
      "batch_size" : 100,
      "n_classes" : 10,
      "n_filters" : [16,32,64,64,128],
      "filter_size" : [3,3,3,3,3],
      "fc_size" : 400,
      "drop_out" : 0.5,
      "augmentation" : 1,
      "batch_normalize" : 1 
  }

  # track this experiment with wandb: all runs will be sent to the given project name
  run = wandb.init(config=config_defaults,job_type='train')
  cfg = wandb.config

  # artifact names
  train_at = os.path.join(PROJECT_NAME, PREFIX_1 + "_train_data_" + str(n_train)) + ":latest"
  val_at = os.path.join(PROJECT_NAME, PREFIX_1 + "_val_data_" + str(n_val)) + ":latest"

  train_data = run.use_artifact(train_at, type='train_data')
  train_dir = train_data.download()
  val_data = run.use_artifact(val_at, type='val_data')
  val_dir = val_data.download()

  # create augmented train and validation data generators
  if cfg.augmentation == 1:
    train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=36,
      width_shift_range=0.2,
      height_shift_range=0.3,
      shear_range=0.3,
      zoom_range=0.4,
      horizontal_flip=True,
      fill_mode='nearest')
  else  :
    train_datagen = ImageDataGenerator(rescale = 1. /255)
  
  val_datagen = ImageDataGenerator(rescale = 1. / 255)

  train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(cfg.img_size, cfg.img_size),
    batch_size=cfg.batch_size,
    class_mode='categorical')

  val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(cfg.img_size, cfg.img_size),
    batch_size=cfg.batch_size,
    class_mode='categorical')
  
  config.n_filters=[np.random.choice(cfg.n_filters_val) for i in range(5)]   #hidden layer sizes array creation

  config.filter_size=[np.random.choice(cfg.filter_size_val) for i in range(5)]

  # instantiate model and callbacks
  model = cnn_model(cfg.img_size,cfg.n_filters,cfg.filter_size,cfg.fc_size,cfg.drop_out,cfg.n_classes)
 
  # train and validate
  history = model.fit(
      train_generator,
      steps_per_epoch = n_train // cfg.batch_size,
      epochs = n_epochs,
      validation_data=val_generator,
      callbacks = [WandbCallback()],
      validation_steps = n_val // cfg.batch_size)

  # save trained model as artifact
  trained_model_artifact = wandb.Artifact(
            MODEL_NAME, type="model",
            description="trained cnn model",
            metadata=dict(cfg))

  model.save(FINAL_MODEL_DIR)
  trained_model_artifact.add_dir(FINAL_MODEL_DIR)
  run.log_artifact(trained_model_artifact)
  run.finish()

In [1]:
train()

NameError: ignored

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

NameError: ignored

In [19]:
def sweeper(sweep_config,PROJECT_NAME):
  sweep_id=wandb.sweep(sweep_config,project=PROJECT_NAME)
  wandb.agent(sweep_id,train,project=PROJECT_NAME)

In [16]:
#sweep dictionary
sweep_config={
    'method':'bayes',
    'metric':{
        'name':'accuracy',
        'goal':'maximize'},

}

parameters_dict={
    #'n_filters':{
    #    'values':[[16,32,64,64,128],[128,64,64,32,16], [32,32,32,32,32]]
    #},
    'n_filters_val':{
        'values':[16,32,64,128]
    },
    #'filter_size':{
    #    'values':[[3,3,3,3,3],[3,3,3,5,5],[5,5,5,3,3]]
    #},
    'filter_size_val':{
        'values':[3,4,5]
    },
    'fc_size':{
        'values':[256,128,512]
    },
    'drop_out':{
        'values':[0.3,0.5,0.8]
    },
    'augmentation':{
      'values':[1,0]  
    },
    'batch_normalize':{
        'values':[1]
    },
    'batch_size':{
        'values':[100]
    },
}

sweep_config['parameters']=parameters_dict

In [17]:
sweep_id=wandb.sweep(sweep_config,project=PROJECT_NAME)

Create sweep with ID: e8pdtbw5
Sweep URL: https://wandb.ai/rohithd/CS6910%20ASSIGNMENT%202/sweeps/e8pdtbw5


In [ ]:
sweeper(sweep_config,PROJECT_NAME)

Create sweep with ID: zbhwwg6q
Sweep URL: https://wandb.ai/rohithd/CS6910%20ASSIGNMENT%202/sweeps/zbhwwg6q


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: mqi1f3wo with config:
wandb: 	augmentation: 1
wandb: 	batch_normalize: 1
wandb: 	batch_size: 100
wandb: 	drop_out: 0.8
wandb: 	fc_size: 256
wandb: 	filter_size: [5, 5, 5, 3, 3]
wandb: 	n_filters: [32, 32, 32, 32, 32]


wandb: Downloading large artifact train_train_data_9000:latest, 2743.84MB. 9000 files... Done. 0:0:0
wandb: Downloading large artifact train_val_data_1000:latest, 300.69MB. 1000 files... Done. 0:0:0


Found 8999 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 225, 225, 32)      2432      
_________________________________________________________________
batch_normalization (BatchNo (None, 225, 225, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 108, 108, 32)      25632     
_________________________________________________________________
batch_normalization_1 (Batch (None, 108, 108, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 32)        0         
____________________________________